# Example of beta reconstruction usage

This notebook gives a basic example of how to do a beta reconstruction analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from defdap.quat import Quat
import defdap.ebsd as ebsd
from defdap.plotting import MapPlot

from beta_reconstruction.reconstruction import (
    do_reconstruction, load_map, assign_beta_variants, 
    construct_variant_map, construct_beta_quat_array, 
    create_beta_ebsd_map)

%matplotlib qt

## Load in EBSD file
The `load_map` function will load the EBSD map from the specified file and
do prerequisite calculations such as segmenting into grains constructing a
neighbour network of how the grains inter-connect.

In [ ]:
ebsd_file_path = "example_data/ZrNb_triplepoint.ctf"

boundary_tolerance = 3
min_grain_size = 3

ebsd_map = load_map(
    ebsd_file_path,
    boundary_tolerance=boundary_tolerance,
    min_grain_size=min_grain_size
)

Set the $\alpha$ and $\beta$ phase ids for the loaded map. Check these with the names of the phases in the EBSD map.

In [ ]:
alpha_phase_id = 0
beta_phase_id = 1

print('alpha phase name: ', ebsd_map.phases[alpha_phase_id].name)
print('beta phase name: ', ebsd_map.phases[beta_phase_id].name)

## Plot EBSD map
Plot the $\alpha$ and $\beta$ grain structure before reconstruction.

In [ ]:
ipf_dir = np.array([0, 0, 1])

ebsd_map.plotIPFMap(ipf_dir, phases=[alpha_phase_id])
ebsd_map.plotIPFMap(ipf_dir, phases=[beta_phase_id])

## Calculate possible beta orientations 

There is a relationship between the $\alpha$ and $\beta$ symmetries, such that for each alpha orientation there are 6 theoretically possible $\beta$ orientations that the $\alpha$ could have transformed from obeying the Burgers relationship.

The set of 6 possible orientations can be narrowed down further (often to a unique solution) by also considering the orientations of neighbouring $\alpha$ grains which are inherited from a single $\beta$ grain. Being inherited from the same $\beta$ grain restricts the possible neighbouring misorientation between different $\alpha$ grains. In reverse, this means that the misorientations can be used to determine the prior $\beta$ orientation.

In [ ]:
do_reconstruction(
    ebsd_map,
    burg_tol=5.,
    ori_tol=3.,
    alpha_phase_id=alpha_phase_id,
    beta_phase_id=beta_phase_id
)

## Find the most common variant for each grain and set this as the beta orientation

Each $\alpha$ grain will now contain an attribute with the number of votes for each of the 6 possible parent $\beta$ orientations or variants. In the simplest interpretation of the variant counts we can consider the orientation of the parent $\beta$ grain to be the mode variant. Where there are two variants with the same count we do not assign the an orientation.

In [ ]:
assign_beta_variants(ebsd_map, "modal", alpha_phase_id=alpha_phase_id)

## Visualise the results

To reconstruct the $\beta$ orientation map, the Burgers transformation associated with the $\beta$ variant identified for each $\alpha$ grain must be applied to each orientation/pixel in that grain. This is done in 2 stages. First a variant map is produced with each grain filled with the index of the identified beta variant (0-5). This variant map is then used with the unique hexagonal symmetries (with respect to the Burgers orientation relation) and the Burgers transformation to transform ever alpha orientation in the map.

In [ ]:
variant_map = construct_variant_map(ebsd_map, alpha_phase_id=alpha_phase_id)
beta_quat_array = construct_beta_quat_array(ebsd_map, variant_map=variant_map)

Create a function to plot an IPF map of the reconstructed $\beta$ orientations and show grains that were not successfully reconstructed in white.

In [ ]:
def plot_beta(variant_map, beta_quat_array, direction, **kwargs):
    beta_IPF_colours = Quat.calcIPFcolours(
        beta_quat_array[variant_map >= 0],
        direction, 
        "cubic"
    ).T

    map_colours = np.zeros(ebsd_map.shape + (3,))
    map_colours[variant_map >= 0] = beta_IPF_colours
    # recolour the -1 and -2 variants
    # -1 grains not succesfully reconstructed (white)
    # -2 clusters too small to be a grain and other phases (black)
    map_colours[variant_map == -1] = np.array([1, 1, 1])
    map_colours[variant_map == -2] = np.array([0, 0, 0])

    return MapPlot.create(ebsd_map, map_colours, **kwargs)

plot_beta(variant_map, beta_quat_array, np.array([0,0,1]))

## Explore the results

The results are stored in the grain objects which comprise the EBSD map.

If running with interactive plots the `locateGrainID()` function should plot an interactive EBSD map. Clicking on a grain will print the grain ID to the notebook cell. This grain ID can then be used to get information about the grain by subsetting the Map object with the grain ID.

In [ ]:
ebsd_map.locateGrainID()

The reconstruction algorithm first calculates the 6 possible $\beta$ orientations for the grain given its mean $\alpha$ orientation. These are stored in the `beta_oris` attribute in the grain.

In [ ]:
grain_id = 8
grain = ebsd_map[grain_id]
grain.beta_oris

The next step considering the misorientation between the grain and its neighbours yields a measure of the deviation from a perfect Burgers transformation if the two grains transformed from the same grain.  The deviation of each vote/neighbour is stored in the `beta_deviations` attribute. A tolerance for the maximum acceptable value of this deviation (`burg_tol`) is set when calling the `do_reconstruction()` method.

In [ ]:
np.array(grain.beta_deviations) *180 /np.pi

The `possible_beta_oris` attribute stores the possible beta orientations of the parent $\beta$ grain as a result of considering the misorientation relation between the neighbouring $\alpha$ grains. There is one list of possible orientations for each neighbour.

In [ ]:
grain.possible_beta_oris

The `possible_beta_oris` are then binned into one of the six `beta_oris` with a tolerance determined by the `ori_tol` variable passed to the `do_reconstruction()` method. This is essentially a vote on which parent $\beta$ grain orientation is most likely. The `variant_count` attribute stores the counts for each possible beta orientation.

In [ ]:
grain.variant_count

Finally, the `locateGrainID` method can be customised to use the `plot_beta` method created earlier, so grains of interest can be selected in the $\beta$ orientation map. Further to this, a custom callback function `click_print_beta_info` can also be passed to `locateGrainID` so that the information printed to screen after clicking a grain can be customised.

In [ ]:
def click_print_beta_info(event, plot):
    if event.inaxes is not plot.ax:
        return
    
    # grain id of selected grain
    currGrainId = int(plot.callingMap.grains[int(event.ydata), int(event.xdata)] - 1)
    if currGrainId < 0:
        return

    # update the grain highlights layer in the plot
    plot.addGrainHighlights([currGrainId], alpha=plot.callingMap.highlightAlpha)
    
    # Print beta info
    grain = plot.callingMap[currGrainId]
    print("Grain ID: {}".format(currGrainId))
    print("Phase name:", grain.phase.name)
    print("Possible beta oris:", grain.possible_beta_oris)
    print("Beta deviations", np.rad2deg(grain.beta_deviations))
    print("Variant count", grain.variant_count)
    print("Assigned variant", grain.assigned_variant)
    print()
    
# Assign the plotting function to use with `locateGrainID`
ebsd_map.plotDefault = plot_beta
                
plot = ebsd_map.locateGrainID(
    variant_map=variant_map, 
    beta_quat_array=beta_quat_array, 
    direction=np.array([0, 0, 1]), 
    clickEvent=click_print_beta_info
)

## Create a new map of the reconstruction and save

Mode can be:
- 'alone': Only include the reconstructed beta
- 'append': Append reconstructed beta to present beta phase
- 'add': Create a new phase for reconstructed beta

In [ ]:
ebsd_map_recon = create_beta_ebsd_map(
    ebsd_map, 
    mode='alone', 
    beta_quat_array=beta_quat_array, 
    variant_map=variant_map, 
    alpha_phase_id=alpha_phase_id, 
    beta_phase_id=beta_phase_id
)

In [ ]:
ebsd_map_recon.save('recon_map')